In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
import numpy as np
import torch
import torch.optim as optim

# Load dataset
data = fetch_california_housing()
X, y = data.data, data.target  # Features and target

# Shuffle and split data
X, y = np.random.permutation(X), np.random.permutation(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [2]:
#created the calibration set and the proper training set
X_cal = X_train[-500:]
y_cal = y_train[-500:]
X_proper_train = X_train[:-500]
y_proper_train = y_train[:-500]

X_proper_train = torch.tensor(X_proper_train, dtype=torch.float32)
y_proper_train = torch.tensor(y_proper_train, dtype=torch.float32).view(-1, 1)

X_cal = torch.tensor(X_cal, dtype=torch.float32)
y_cal = torch.tensor(y_cal, dtype=torch.float32).view(-1, 1)

X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)


In [3]:
def quantile_loss(y_pred, y_true, tau):
    errors = y_true - y_pred
    return torch.mean(torch.max(tau * errors, (tau - 1) * errors))


In [4]:
# Model parameters: Weight and Bias
input_dim = X_proper_train.shape[1]
W_lower = torch.randn((input_dim, 1), requires_grad=True, dtype=torch.float32)
b_lower = torch.zeros(1, requires_grad=True, dtype=torch.float32)

W_upper = torch.randn((input_dim, 1), requires_grad=True, dtype=torch.float32)
b_upper = torch.zeros(1, requires_grad=True, dtype=torch.float32)


In [ ]:
# Hyperparameters
alpha = 0.01  # 90% Prediction Interval (5% in each tail)
taus = [alpha / 2, 1 - alpha / 2]  # Lower and Upper Quantiles
epochs = 500
lr = 0.01

# Optimizers
opt_lower = optim.Adam([W_lower, b_lower], lr=lr)
opt_upper = optim.Adam([W_upper, b_upper], lr=lr)

for epoch in range(epochs):
    # Lower Quantile Model (tau = alpha/2)
    opt_lower.zero_grad()
    y_pred_lower = torch.matmul(X_proper_train, W_lower) + b_lower
    loss_lower = quantile_loss(y_pred_lower, y_proper_train, taus[0])
    loss_lower.backward()
    opt_lower.step()

    # Upper Quantile Model (tau = 1-alpha/2)
    opt_upper.zero_grad()
    y_pred_upper = torch.matmul(X_proper_train, W_upper) + b_upper
    loss_upper = quantile_loss(y_pred_upper, y_proper_train, taus[1])
    loss_upper.backward()
    opt_upper.step()

print("Training Complete")


Training Complete


In [6]:
y_lower_pred = torch.matmul(X_cal, W_lower).detach().numpy().flatten()
y_upper_pred = torch.matmul(X_cal, W_upper).detach().numpy().flatten()

y_lower_pred = torch.tensor(y_lower_pred)
y_upper_pred = torch.tensor(y_upper_pred)
y_cal = torch.tensor(y_cal)

score_array = torch.maximum(y_lower_pred - y_cal, y_cal - y_upper_pred)

/tmp/ipykernel_91500/2915519900.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_cal = torch.tensor(y_cal)


In [7]:
# Compute the quantile index
import math
n = score_array.shape[0]
quantile_index = math.ceil((n + 1) * (1 - alpha)) / n

# Compute the required quantile
q_hat = torch.quantile(score_array, quantile_index)

In [8]:
y_lower_pred_test = torch.matmul(torch.tensor(X_test, dtype=torch.float32), W_lower).detach().numpy().flatten()
y_upper_pred_test = torch.matmul(torch.tensor(X_test, dtype=torch.float32), W_upper).detach().numpy().flatten()
y_lower_pred_test = torch.tensor(y_lower_pred_test)
y_upper_pred_test = torch.tensor(y_upper_pred_test)


/tmp/ipykernel_91500/2180497115.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_lower_pred_test = torch.matmul(torch.tensor(X_test, dtype=torch.float32), W_lower).detach().numpy().flatten()
/tmp/ipykernel_91500/2180497115.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_upper_pred_test = torch.matmul(torch.tensor(X_test, dtype=torch.float32), W_upper).detach().numpy().flatten()


In [9]:

covered = (y_test >= (y_lower_pred_test - q_hat)) & (y_test <= (y_upper_pred_test + q_hat))

coverage_percentage = covered.float().mean().item() * 100

print(f"Coverage Percentage: {coverage_percentage}%")

Coverage Percentage: 52.424198389053345%


In [10]:
1-alpha + 1 / (n+1)

0.501996007984032